# Weight Initialization
## Xavier / He Initialization
### 1) import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.3.0


### 2) Checkpoint Function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

### 3) Data load & pre-processing function

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

### 4) Performance Function

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

### 5) Model Function

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

### 6) Create Model(function version)
- weight_init를 `tf.keras.initializers.glorot_uniform()`으로 설정한다.
- he initialization(ReLU에 특화)을 이용하고 싶으면 `tf.keras.initializers.he_uniform()`을 쓰면 된다.
- class version의 model은 lec10-1_MNIST_nn_ReLU_vs_softmax.ipynb 참고

In [6]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4) :
        model.add(dense(512, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

### 7) Define data & hyper-parameter

In [7]:
# dataset
train_x, train_y, test_x, test_y = load_mnist()

# parameters
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

# Graph Input using Dataset API
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

### 8) Define Model & Optimizer & Writer

In [8]:
# Model
network = create_model_function(label_dim)

# Training
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Writer
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_xavier'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

### 9) Restore checkpoint & start train or test phase

In [9]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 1.6260, train_loss: 2.02774239, train_accuracy: 0.3984, test_Accuracy: 0.2192
Epoch: [ 0] [    1/  468] time: 2.6270, train_loss: 2.00318575, train_accuracy: 0.3125, test_Accuracy: 0.3076
Epoch: [ 0] [    2/  468] time: 3.5370, train_loss: 1.77012730, train_accuracy: 0.6172, test_Accuracy: 0.5544
Epoch: [ 0] [    3/  468] time: 4.4590, train_loss: 1.58321738, train_accuracy: 0.7109, test_Accuracy: 0.6605
Epoch: [ 0] [    4/  468] time: 5.5240, train_loss: 1.32263064, train_accuracy: 0.7188, test_Accuracy: 0.6906
Epoch: [ 0] [    5/  468] time: 6.5595, train_loss: 1.02093220, train_accuracy: 0.7734, test_Accuracy: 0.7538
Epoch: [ 0] [    6/  468] time: 7.5465, train_loss: 0.90461159, train_accuracy: 0.7500, test_Accuracy: 0.7358
Epoch: [ 0] [    7/  468] time: 8.4995, train_loss: 0.65332967, train_accuracy: 0.8594, test_Accuracy: 0.7676
Epoch: [ 0] [    8/  468] time: 9.4315,

Epoch: [ 0] [   74/  468] time: 70.0526, train_loss: 0.20129794, train_accuracy: 0.9141, test_Accuracy: 0.9248
Epoch: [ 0] [   75/  468] time: 70.8456, train_loss: 0.15693994, train_accuracy: 0.9531, test_Accuracy: 0.9268
Epoch: [ 0] [   76/  468] time: 71.6375, train_loss: 0.19360539, train_accuracy: 0.9297, test_Accuracy: 0.9283
Epoch: [ 0] [   77/  468] time: 72.4526, train_loss: 0.18039823, train_accuracy: 0.9609, test_Accuracy: 0.9236
Epoch: [ 0] [   78/  468] time: 73.2466, train_loss: 0.18663560, train_accuracy: 0.9219, test_Accuracy: 0.9260
Epoch: [ 0] [   79/  468] time: 74.0426, train_loss: 0.18489511, train_accuracy: 0.9531, test_Accuracy: 0.9299
Epoch: [ 0] [   80/  468] time: 74.8356, train_loss: 0.12880608, train_accuracy: 0.9688, test_Accuracy: 0.9322
Epoch: [ 0] [   81/  468] time: 75.6556, train_loss: 0.19755155, train_accuracy: 0.9375, test_Accuracy: 0.9310
Epoch: [ 0] [   82/  468] time: 76.4516, train_loss: 0.26066518, train_accuracy: 0.9375, test_Accuracy: 0.9364
E

Epoch: [ 0] [  148/  468] time: 129.5045, train_loss: 0.17099151, train_accuracy: 0.9766, test_Accuracy: 0.9462
Epoch: [ 0] [  149/  468] time: 130.2945, train_loss: 0.09303920, train_accuracy: 0.9609, test_Accuracy: 0.9456
Epoch: [ 0] [  150/  468] time: 131.0876, train_loss: 0.08633296, train_accuracy: 0.9844, test_Accuracy: 0.9421
Epoch: [ 0] [  151/  468] time: 131.8876, train_loss: 0.12712811, train_accuracy: 0.9531, test_Accuracy: 0.9438
Epoch: [ 0] [  152/  468] time: 132.6985, train_loss: 0.12477614, train_accuracy: 0.9609, test_Accuracy: 0.9486
Epoch: [ 0] [  153/  468] time: 133.4925, train_loss: 0.15017217, train_accuracy: 0.9688, test_Accuracy: 0.9502
Epoch: [ 0] [  154/  468] time: 134.2826, train_loss: 0.11167154, train_accuracy: 0.9531, test_Accuracy: 0.9500
Epoch: [ 0] [  155/  468] time: 135.0926, train_loss: 0.23784347, train_accuracy: 0.9297, test_Accuracy: 0.9475
Epoch: [ 0] [  156/  468] time: 135.9165, train_loss: 0.13508548, train_accuracy: 0.9609, test_Accuracy:

Epoch: [ 0] [  222/  468] time: 189.4215, train_loss: 0.13759083, train_accuracy: 0.9609, test_Accuracy: 0.9558
Epoch: [ 0] [  223/  468] time: 190.2706, train_loss: 0.07331480, train_accuracy: 0.9688, test_Accuracy: 0.9552
Epoch: [ 0] [  224/  468] time: 191.1875, train_loss: 0.11901888, train_accuracy: 0.9531, test_Accuracy: 0.9577
Epoch: [ 0] [  225/  468] time: 192.1646, train_loss: 0.17313038, train_accuracy: 0.9531, test_Accuracy: 0.9607
Epoch: [ 0] [  226/  468] time: 192.9916, train_loss: 0.08162038, train_accuracy: 0.9766, test_Accuracy: 0.9598
Epoch: [ 0] [  227/  468] time: 193.7895, train_loss: 0.07334559, train_accuracy: 0.9766, test_Accuracy: 0.9596
Epoch: [ 0] [  228/  468] time: 194.6136, train_loss: 0.14442682, train_accuracy: 0.9688, test_Accuracy: 0.9606
Epoch: [ 0] [  229/  468] time: 195.4666, train_loss: 0.12607107, train_accuracy: 0.9531, test_Accuracy: 0.9606
Epoch: [ 0] [  230/  468] time: 196.4136, train_loss: 0.05815840, train_accuracy: 0.9766, test_Accuracy:

Epoch: [ 0] [  296/  468] time: 251.5745, train_loss: 0.04763221, train_accuracy: 0.9844, test_Accuracy: 0.9650
Epoch: [ 0] [  297/  468] time: 252.4235, train_loss: 0.03990950, train_accuracy: 0.9844, test_Accuracy: 0.9628
Epoch: [ 0] [  298/  468] time: 253.2576, train_loss: 0.08537704, train_accuracy: 0.9688, test_Accuracy: 0.9633
Epoch: [ 0] [  299/  468] time: 254.1295, train_loss: 0.04370024, train_accuracy: 0.9844, test_Accuracy: 0.9611
Epoch: [ 0] [  300/  468] time: 254.9535, train_loss: 0.07159045, train_accuracy: 0.9688, test_Accuracy: 0.9620
Epoch: [ 0] [  301/  468] time: 255.7796, train_loss: 0.08365688, train_accuracy: 0.9609, test_Accuracy: 0.9633
Epoch: [ 0] [  302/  468] time: 256.5996, train_loss: 0.15267873, train_accuracy: 0.9453, test_Accuracy: 0.9654
Epoch: [ 0] [  303/  468] time: 257.3975, train_loss: 0.09153818, train_accuracy: 0.9688, test_Accuracy: 0.9641
Epoch: [ 0] [  304/  468] time: 258.2006, train_loss: 0.12802383, train_accuracy: 0.9609, test_Accuracy:

Epoch: [ 0] [  370/  468] time: 315.2236, train_loss: 0.19611388, train_accuracy: 0.9766, test_Accuracy: 0.9615
Epoch: [ 0] [  371/  468] time: 316.0595, train_loss: 0.12461497, train_accuracy: 0.9375, test_Accuracy: 0.9617
Epoch: [ 0] [  372/  468] time: 316.8956, train_loss: 0.16707510, train_accuracy: 0.9453, test_Accuracy: 0.9628
Epoch: [ 0] [  373/  468] time: 317.7075, train_loss: 0.07825755, train_accuracy: 0.9766, test_Accuracy: 0.9618
Epoch: [ 0] [  374/  468] time: 318.5296, train_loss: 0.11893717, train_accuracy: 0.9688, test_Accuracy: 0.9596
Epoch: [ 0] [  375/  468] time: 319.3986, train_loss: 0.07251228, train_accuracy: 0.9766, test_Accuracy: 0.9583
Epoch: [ 0] [  376/  468] time: 320.2166, train_loss: 0.14144793, train_accuracy: 0.9609, test_Accuracy: 0.9593
Epoch: [ 0] [  377/  468] time: 321.0805, train_loss: 0.10153481, train_accuracy: 0.9609, test_Accuracy: 0.9619
Epoch: [ 0] [  378/  468] time: 322.0676, train_loss: 0.13756098, train_accuracy: 0.9531, test_Accuracy:

Epoch: [ 0] [  444/  468] time: 386.9306, train_loss: 0.07798380, train_accuracy: 0.9766, test_Accuracy: 0.9563
Epoch: [ 0] [  445/  468] time: 387.7666, train_loss: 0.07773786, train_accuracy: 0.9688, test_Accuracy: 0.9585
Epoch: [ 0] [  446/  468] time: 388.5606, train_loss: 0.10789034, train_accuracy: 0.9688, test_Accuracy: 0.9590
Epoch: [ 0] [  447/  468] time: 389.4045, train_loss: 0.04355256, train_accuracy: 0.9844, test_Accuracy: 0.9562
Epoch: [ 0] [  448/  468] time: 390.1996, train_loss: 0.10220531, train_accuracy: 0.9766, test_Accuracy: 0.9558
Epoch: [ 0] [  449/  468] time: 391.0566, train_loss: 0.12730236, train_accuracy: 0.9531, test_Accuracy: 0.9567
Epoch: [ 0] [  450/  468] time: 391.8825, train_loss: 0.05327401, train_accuracy: 0.9844, test_Accuracy: 0.9566
Epoch: [ 0] [  451/  468] time: 392.7056, train_loss: 0.03150612, train_accuracy: 1.0000, test_Accuracy: 0.9585
Epoch: [ 0] [  452/  468] time: 393.5476, train_loss: 0.05335645, train_accuracy: 0.9844, test_Accuracy:

## 3. 결론
> Weight Initialization 기법을 쓰면, 좀 더 높은 accuracy의 모델을 만들 수 있다.

### ReLU + RandomNormal
- Test accuracy : 96.69%

### ReLU + Xavier
- Test accuracy : 96.73%

### ReLU + He Initialization
- Test accuracy : 96.88% (강의 자료에 의해)
- 코드는 생략